# Download our content for model

In [ ]:
url = 'https://storage.googleapis.com/datascience-materials/dogs-vs-cats.zip'

import requests
import zipfile
from io import BytesIO

extract_to_directory = "extracted_content" # Directory to extract files into

# Send GET request and wrap content in BytesIO
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as zfile:
    # Extract all files to a specified directory
    zfile.extractall(extract_to_directory)

print(f"Content extracted to '{extract_to_directory}' successfully.")

In [ ]:
#!pip install tensorflow

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
# make train, test, and validation sets
img_size = (224,224)
batch_size = 32


train_ds = keras.utils.image_dataset_from_directory(
    "extracted_content/dogs-vs-cats/train/",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary",
    shuffle=True,
)

val_ds = keras.utils.image_dataset_from_directory(
    "extracted_content/dogs-vs-cats/val/",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary",
    shuffle=True,
)

test_ds = keras.utils.image_dataset_from_directory(
    "extracted_content/dogs-vs-cats/test/",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary",
    shuffle=True,
)



Found 20000 files belonging to 2 classes.
Found 2500 files belonging to 2 classes.
Found 2500 files belonging to 2 classes.


In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense #Load pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze weights

# Add new layers
model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
])

# Due to computational constraints, a representative subset of the training data was used for optimization.

In [4]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Expecting 0.0 - 255.0 min/max format
for x, y in train_ds.take(1):
    print(tf.reduce_min(x).numpy(), tf.reduce_max(x).numpy())

0.0 255.0


In [6]:
TRAIN_STEPS = 200
VAL_STEPS = 50

Build our model checkpoints and early stopping

In [7]:
checkpoint = ModelCheckpoint(
    filepath="best_vgg16_model.keras",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True,
    verbose=1
)

Compile our sequential model for loss of Binary Cross Entropy

In [8]:
model.compile(optimizer="adam", loss = BinaryCrossentropy(from_logits=False), metrics= ["accuracy"])

Fit model on our params, including callbacks

In [9]:
model.fit(
    train_ds.take(TRAIN_STEPS),
    epochs=10,
    validation_data=val_ds.take(VAL_STEPS),
    callbacks=[checkpoint, early_stop]
)

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8273 - loss: 6.8858
Epoch 1: val_loss improved from None to 0.12685, saving model to best_vgg16_model.keras

Epoch 1: finished saving model to best_vgg16_model.keras
200/200 ━━━━━━━━━━━━━━━━━━━━ 867s 4s/step - accuracy: 0.9159 - loss: 1.5772 - val_accuracy: 0.9613 - val_loss: 0.1268
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9733 - loss: 0.0848
Epoch 2: val_loss did not improve from 0.12685
200/200 ━━━━━━━━━━━━━━━━━━━━ 892s 4s/step - accuracy: 0.9800 - loss: 0.0615 - val_accuracy: 0.9688 - val_loss: 0.1371
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9908 - loss: 0.0284
Epoch 3: val_loss improved from 0.12685 to 0.12134, saving model to best_vgg16_model.keras

Epoch 3: finished saving model to best_vgg16_model.keras
200/200 ━━━━━━━━━━━━━━━━━━━━ 863s 4s/step - accuracy: 0.9928 - loss: 0.0196 - val_accuracy: 0.9694 - val_loss: 0.1213
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/ste

In [ ]:
from tensorflow.keras.models import load_model

Load our best model from our training

In [ ]:
best_model = load_model("best_vgg16_model.keras")

Make predictions on our test dataset

In [ ]:
y_pred_probs = best_model.predict(test_ds)
y_pred = (y_pred_probs > 0.5).astype(int)

In [ ]:
print(y_pred_probs.shape)
print(y_pred.shape)

In [ ]:
import numpy as np

y_true = np.concatenate([
    y for x, y in test_ds
]).astype(int)


In [ ]:
y_true = y_true.flatten()
y_pred = y_pred.flatten()

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)

Despite preprocessing and optimization using EarlyStopping and ModelCheckpoint, the frozen VGG16 backbone with limited training iterations produced near-random performance. This outcome highlights the computational trade-offs inherent in transfer learning and demonstrates that model capacity and fine-tuning are critical when adapting large pre-trained networks under constrained resources.